# Implementing motion detection to find Kodak in a video frame
Once i have code that can detect Kodak in a video, I'll connect it to the camera. 

In [1]:
# Importing packages
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from numpy import moveaxis
import pandas as pd
import cv2

import subprocess
import IPython.display as ipd

import torch
from torchvision import models
import torch.nn as nn

import torch
from torchvision import models, transforms
from PIL import Image

In [2]:
# test1_input = "test1.MOV"
# test2_in = "test2.MOV"

In [3]:
# Converting from .mov to .mp4 and removing the last second of the video
# Not needed after you do it once!
# subprocess.run(['ffmpeg', '-i', test2_in, '-qscale', '0', 'test2.mp4', '-loglevel', 'quiet', ])

## Ok, lets do the motion detector now
If a moving object appears in 5 consecutive frames, I want the box to be passed to my pre-trained kodak_detector.

In [4]:
# frame processing to feed a frame into the CNN
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256, antialias=True),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# Preparing for the video
class_names = [0, 1]
motion_frames = 0
consecutive_frames = 20
tracker = None
detected = False
triggered = False

In [6]:
# Defining model architecture
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

In [7]:
# Loading model weights
model.load_state_dict(torch.load('is_that_kodak.pth'))

<All keys matched successfully>

In [8]:
def is_it_kodak(frame):
    # Turning on the model
    model.eval()

    # transforming the frame
    frame_proc = preprocess(frame)
    squeezed = torch.unsqueeze(frame_proc, 0)

    with torch.no_grad():
        prediction = model(squeezed)

    pos_or_neg = np.argmax(prediction, axis = 1)[0] == 1
    return pos_or_neg

In [9]:
test1 = cv2.VideoCapture('test2.mp4')

In [10]:
check, frame = test1.read()
if not check:
    exit()

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (41, 41), 0)

background = gray

# Create a ROI for spray zone
spray_zone = np.zeros_like(gray)
im_h, im_w = gray.shape

#spray zone top left
sz_tl_x = round(0.25 * im_w)
sz_tl_y = round(0.25 * im_h)

#spray zone width and height
sz_w = round(0.5 * im_w)
sz_h = round(0.5 * im_h)

spray_zone[sz_tl_y : sz_tl_y + sz_h, sz_tl_x : sz_tl_x + sz_w] = 255

In [11]:
while True:
    # Take in first frame, convert it to gray scale, then blur it
    check, frame = test1.read()
    if not check:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (41, 41), 0)

    # If we've already identified the cat, there's no need to do motion detection
    if detected:
        if not triggered:
            success, box = tracker.update(frame)
            if success:
                (x, y, w, h) = [int(i) for i in box]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 5)
                top_left = (x, y)
                bottom_right = (x + w, y + h)
                # Calculate the coordinates of the corners of the spray zone
                sz_top_left = (sz_tl_x, sz_tl_y)
                sz_bottom_right = (sz_tl_x + sz_w, sz_tl_y + sz_h)
                # Check if any of the corners of the tracking box are inside the spray zone
                if (sz_top_left[0] < top_left[0] < sz_bottom_right[0] or
                    sz_top_left[0] < bottom_right[0] < sz_bottom_right[0]) and \
                (sz_top_left[1] < top_left[1] < sz_bottom_right[1] or
                    sz_top_left[1] < bottom_right[1] < sz_bottom_right[1]):
                #if x < sz_tl_x + sz_w < x + w and y < sz_tl_y + sz_h < y + h:
                    print("Alert, alert. There is a small criminal present. Triggering deterrant system.")
                    # Raspberry Pi trigger code
                    triggered = True
    else:
        # If there is not yet a background, make one
        if background is None:
            background = gray
            continue
        
        # Finding the difference between background and current frame
        diff_frame = cv2.absdiff(background, gray)

        # If the difference is greater than 50, make a binary mask where the motion is white
        threshold = cv2.threshold(diff_frame, 100, 255, cv2.THRESH_BINARY)[1]
        threshold = cv2.dilate(threshold, None, iterations = 2)

        # Given motion, find the contours of the object
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        motion_detected = False

        for contour in contours:
            if cv2.contourArea(contour) < 10000:
                continue
            motion_detected = True
            (x, y, w, h) = cv2.boundingRect(contour)
            # Drawing a box around the moving object
            cv2.rectangle(frame, (x - 20, y - 20), (x + w + 20, y + h + 20), (0, 0, 255), 3)
    
        if motion_detected:
            motion_frames += 1
        else:
            motion_frames = 0

        # If motion is detected in 20 consecutive frames, save the frame and send to model
        if motion_frames >= consecutive_frames:
            frame_to_identify = frame[y:y + h, x:x + w]
            if is_it_kodak(frame_to_identify):
                    print("CAT DETECTED OMG!!!!")
                    cv2.imwrite('detected_cat.png', frame_to_identify)
                    detected = True
                    tracker = cv2.TrackerMIL_create()
                    tracker.init(frame, (x, y, w, h))
  
    cv2.rectangle(frame, (sz_tl_x, sz_tl_y), (sz_tl_x + sz_w, sz_tl_y + sz_h), (0, 255, 0), 3)

    # Displaying color frame with contour of motion of object 
    cv2.imshow("Color Frame", frame) 
  
    key = cv2.waitKey(1) 
    # if q entered whole process will stop 
    if key == ord('q'):
        break

CAT DETECTED OMG!!!!
Alert, alert. There is a small criminal present. Triggering deterrant system.


In [12]:
test1.release()
cv2.destroyAllWindows()